In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from config import conf

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/중도탈락 학생 현황 (대학)_2026-01-14175237927.csv" #
df = pd.read_csv(file_path, encoding='utf-8')  # 'cp949')

print("데이터 로드 완료")
display(df.head()) 
print(len(df))

# 3. 모델 및 에이전트 설정
model_name = "gemma3:27b"  # "gemma3:27b"

llm = OllamaLLM(model=model_name, 
                temperature=0,
                # 모델이 한 번의 응답에서 생성할 최대 토큰 수 (설명 길이를 강제 제한) 길게 설명하는 것을 방지
                num_predict=2048,  # # 512,  # 이상한 소리하면 줄여보기
               )
llm_with_stop = llm.bind(stop=["Observation:", "\nObservation", "Thought:", "\n\tObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = conf.CUSTOM_PREFIX

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력 (custom_prefix, agent_executor_kwargs, full_query 등 tuning 위한 debugger로 활용
    allow_dangerous_code=True,  # 코드 실행 허용
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리 (에러 발생 시 모델에게 던져줄 가이드 문구)
    agent_executor_kwargs={
        "handle_parsing_errors": conf.HANDLE_PARSING_ERRORS,
    },
    agent_type="zero-shot-react-description",
    max_iterations=100, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 통계값 요약해줘"
hypo_req = "학업중단 사유별 학업중단율을 비교하고 가설을 설정하여라"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.
""" + conf.FULL_QUERY

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.run(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")